In [1]:
# some scripts to auto-generate experiment specification files
# we want to auto-generate a directory of files that can be fed into runBatch.sh
# uses "template" directory to get onAndroid-bin scripts

# need to make exp.xml
# and then make onAndroid// directory, with separate run_1_gcn.sh script for each node
# onAndroid-bin can be copied directly from template

# IMPORTANT!
# after loading onto the server, need to run:
# $ find ./ -name "*.sh" -exec chmod +x {} \;
# so that all the bash files are actually executable by the experiment
# otherwise, experiment will run empty, with no scripts being executed!

In [2]:
import os, shutil, stat

In [3]:
def makefiletree(experimentname, numnodes):
    '''make file directory for new experiment spec
    experimentname is the experiment directory
    numnodes is the number of nodes in the experiment
    this will fail if a file with this name already exists
    returns the path to the directory created'''
    
    # make top level directory
    exppath = os.path.join(r'experiments/', experimentname)
    os.makedirs(exppath)
    
    # make a file for each node, in an onAndroid sub-directory
    onAndroidpath = os.path.join(exppath, 'onAndroid')
    os.makedirs(onAndroidpath)
    
    for n in xrange(numnodes):
        nodepath = os.path.join(onAndroidpath, 'node' + '{n:03d}'.format(n = n+1))
        os.makedirs(nodepath)
    
    # make onAndroid-bin directory, copied from template file
    shutil.copytree(r'experiments/template/onAndroid-bin', os.path.join(exppath, 'onAndroid-bin'))
    
    return exppath

In [4]:
def makeexp(exppath, numnodes, startdelay, duration):
    '''make exp.xml file for an experiment, assuming makefiletree() has been run already
    exppath: path to experiment directory - output from makefiletree()
    numnodes: number of nodes in the experiment
    startdelay: amount of time to wait before starting experiment (in seconds)
    duration: duration of the experiment (in seconds)'''
    
    # xml header
    header = '''<?xml version="1.0" encoding="UTF-8"?>
<Exp>
  <NodeInfo>
'''
    
    # spec each node
    nodes = ''

    for n in xrange(numnodes):

        name = 'node' + '{n:03d}'.format(n = n+1)
        mgmt_ip = '192.168.1.{}'.format(n+1)
        interface_ip = '10.1.1.{}'.format(n+1)

        newnode = '''    <Node name="{}">
      <MgmtNetIP>{}/24</MgmtNetIP>
      <MgmtNetBroadcastIP>192.168.1.255</MgmtNetBroadcastIP>
      <Interface name="wlan0" ip="{}/24" />
    </Node>
'''.format(name, mgmt_ip, interface_ip)
        
        nodes += newnode
    
    # footer with experiment startdelay and duration

    footer = '''  </NodeInfo>
  <EmuInfo>
    <Emu name="controller">
      <MgmtNetIP>192.168.1.201/24</MgmtNetIP>
      <MgmtVLAN>None</MgmtVLAN>
      <MgmtBridge>br0</MgmtBridge>
    </Emu>
  </EmuInfo>
  <Dirs>
    <HarnessLogDir>./log</HarnessLogDir>
    <ControllerDataDir>./data</ControllerDataDir>
    <ControllerCollectDir>./collect</ControllerCollectDir>
    <NodeExpRoot>/sdcard/llcbmen</NodeExpRoot>
    <NodeBinDir>/data/llcbmen</NodeBinDir>
  </Dirs>
  <Collector servername="controller" port="12377" />
  <Steps>
    <ConfigWithMgmt pause="error" />
    <UberLocalStartup beaconerInterval="10000" startWait="{}" duration="{}" pause="error" />
  </Steps>
</Exp>
'''.format(startdelay, duration)
    
    
    # write to file
    exp = os.path.join(exppath, 'exp.xml')
    
    with open(exp, 'wb') as f:
        document = header + nodes + footer
        f.write(document)

In [5]:
def makenodescripts(exppath, numnodes, duration, rate, size):
    '''make run_1_gcn.sh files for an experiment, assuming makefiletree() has been run already
    exppath: path to experiment directory - output from makefiletree()
    numnodes: number of nodes in the experiment
    duration: duration of the experiment (in seconds)
    rate: average rate of poisson process to run, in packets/second
    size: size of packets to send, in bytes'''
    
        
    header = """#!/system/xbin/bash -login

nodeName=$1
nodeNum=$2
runid=$3
expRoot=$4
expName=$5
basetime=$6

dataFile=$expRoot/data/$runid/data/$nodeName/gcn.llg
mgenLogFile=$expRoot/data/$runid/data/$nodeName/mgen.log
olsrLogFile=$expRoot/data/$runid/data/$nodeName/olsr.log

echo '''
0.0 LISTEN UDP 5001
"""
    
    footer = """''' > /sdcard/exp.mgen

echo "Starting mgen."
olsrd -d 0 -i wlan0 > $olsrLogFile
sleep 1
mgen input /sdcard/exp.mgen > $mgenLogFile
sleep 1
exit 0
"""
    
    
    # for each node....
    
    for n in xrange(numnodes):
        mgenspec = ''

        # declare mgen script
        for dest in xrange(numnodes): # send traffic to all other nodes, but not to yourself
            if n == dest:
                pass
            else:
                mgenspec += '60.0 ON {} UDP SRC 5001 DST 10.1.1.{}/5001 POISSON [{} {}] INTERFACE wlan0\n'.format(dest+1, dest+1, rate, size)
        
        # stop flows
        for dest in xrange(numnodes):
            if n == dest:
                pass
            else:
                mgenspec += '{}.0 OFF {}\n'.format(duration, dest+1)
    
        # write to file
        document = header + mgenspec + footer
        
        nodepath = os.path.join(exppath, 'onAndroid', 'node' + '{n:03d}'.format(n = n+1), 'run_1_gcn.sh')
        with open(nodepath, 'wb') as f:
            f.write(document)
            
        # mark run_1_gcn.sh as an executable file - equivalent of chmod +x in linux
        executable_by_all = stat.S_IXUSR | stat.S_IXGRP | stat.S_IXOTH
        os.chmod(nodepath, os.stat(nodepath).st_mode | executable_by_all)

In [24]:
# example run

experimentname = 'n30_all_calibration_08'         # name of experiment
numnodes = 30                                   # number of nodes in experiment
startdelay = 30                                # duration of startWait in seconds
duration = 300                                 # duration of experiment in seconds
rate = 0.8                                       # number of packets to attempt to send per second per destination
size = 128                                     # bytes per packet

In [25]:
exppath = makefiletree(experimentname, numnodes)
makeexp(exppath, numnodes, startdelay, duration)
makenodescripts(exppath, numnodes, duration, rate, size)